In [11]:
import pandas as pd
import numpy as np

pop_df = pd.read_csv("population_utf8.csv", sep = ";", header = None)
pop_df.columns = ["municipality", "year", "population", "num_men", "num_women"]
pop_df = pop_df[["municipality", "population"]]

# 0 population rows considered as missing data and will be removed
pop_df = pop_df.replace(0, np.nan)
pop_df = pop_df.dropna(axis = 0)

# some names have format: "Ähtäri;Etseri"
def remove_altname(string):
    return string.split(";")[0]

pop_df["municipality"] = pop_df["municipality"].apply(remove_altname)

pop_df

,municipality,population
0,Akaa,17052.0
2,Alajärvi,10171.0
4,Alavieska,2687.0
5,Alavus,12103.0
8,Asikkala,8374.0
...,...,...
448,Ylivieska,14976.0
451,Ylöjärvi,32260.0
452,Ypäjä,2468.0
454,Ähtäri,6178.0


In [14]:
speed_df = pd.read_csv("results-2019-01-nopeustesti.csv", skiprows = [0], header = None)
speed_df.columns = ["time", "country", "area", "municipality", "down_speed", "up_speed", "latency", "jitter"]
speed_df = speed_df[["municipality", "down_speed", "up_speed"]]
speed_df

,municipality,down_speed,up_speed
0,Turku,3601.0,4380.0
1,Mäntsälä,5688.0,9820.0
2,Raisio,19428.0,4657.0
3,Mäntsälä,9522.0,8207.0
4,Kerava,90066.0,9215.0
...,...,...,...
98318,Lahti,21101.0,4938.0
98319,Turku,32848.0,17558.0
98320,Kuusamo,55308.0,69665.0
98321,Helsinki,5275.0,3271.0


In [22]:
speed_df.groupby("municipality").mean()

,down_speed,up_speed
municipality,,
AEetsae,50578.000000,7025.000000
AEijaenneva,11579.000000,6393.750000
AEitsaari,58533.545455,35942.212121
AEystoe,28686.000000,17566.000000
Ahmovaara,48836.200000,7352.800000
...,...,...
Ylöjärvi,32280.376623,9848.155844
Ypaejae,39093.966667,18850.588889
Ytteresse,43410.000000,26556.000000


In [48]:
combined_df = pd.DataFrame()
combined_df["DL avg"] = speed_df.groupby("municipality")["down_speed"].mean()
combined_df["DL var"] = speed_df.groupby("municipality")["down_speed"].var()
combined_df["UL avg"] = speed_df.groupby("municipality")["up_speed"].mean()
combined_df["UL var"] = speed_df.groupby("municipality")["up_speed"].var()

combined_df.reset_index(level = 0, inplace = True)

# above-mentioned operations transform "ä" to "ae" and "ö" to "oe"
def to_finnish(string):
    if string is not "Joensuu":
        string = string.replace("AE", "Ä")
        string = string.replace("ae", "ä")
        string = string.replace("OE", "Ö")
        string = string.replace("oe", "ö")
    return string

combined_df["municipality"] = combined_df["municipality"].apply(to_finnish)

# if there is only one observation from municipality, variance is NA
combined_df = combined_df.dropna(axis = 0)

combined_df

,municipality,DL avg,DL var,UL avg,UL var
1,Äijänneva,11579.000000,4.512437e+06,6393.750000,3.223109e+05
2,Äitsaari,58533.545455,1.764261e+09,35942.212121,1.062507e+09
4,Ahmovaara,48836.200000,6.642606e+08,7352.800000,1.852068e+07
5,Ahvio,17033.800000,1.720168e+08,2427.400000,9.766728e+06
6,Airaksela,44399.923077,8.839720e+08,2867.205128,7.648711e+06
...,...,...,...,...,...
1000,Ylöjärvi,32280.376623,1.538512e+09,9848.155844,3.034912e+08
1001,Ypäjä,39093.966667,1.001320e+09,18850.588889,2.383602e+08
1002,Ytteresse,43410.000000,5.030792e+06,26556.000000,1.180008e+07
1003,Ähtäri,28637.454545,7.146832e+08,21013.363636,8.321656e+08


In [49]:
# only 222 municipalities (out of 320) are found in both dataframes
combined_df = pd.merge(combined_df, pop_df, how = "inner", on = "municipality")
combined_df = combined_df.iloc[:, [0, -1, 1, 2, 3, 4]]
combined_df

,municipality,population,DL avg,DL var,UL avg,UL var
0,Akaa,17052.0,37535.521739,2.982847e+09,14488.376812,8.934048e+08
1,Alajärvi,10171.0,45620.145833,9.584660e+08,11303.937500,7.720442e+07
2,Alavus,12103.0,55538.953271,2.165521e+09,33308.925234,1.212724e+09
3,Askola,5064.0,43079.666667,3.040476e+09,8556.666667,5.917759e+07
4,Aura,3982.0,27335.247706,5.459267e+08,6654.616822,4.910725e+07
...,...,...,...,...,...,...
217,Ylivieska,14976.0,22256.431373,4.288729e+08,5934.524752,4.715238e+08
218,Ylöjärvi,32260.0,32280.376623,1.538512e+09,9848.155844,3.034912e+08
219,Ypäjä,2468.0,39093.966667,1.001320e+09,18850.588889,2.383602e+08
220,Ähtäri,6178.0,28637.454545,7.146832e+08,21013.363636,8.321656e+08


In [50]:
import json

combined_df.to_json("netspeeds.json", orient = "records")

parsed = json.load(open("netspeeds.json"))
print(json.dumps(parsed[2:5], indent = 2))

[
  {
    "municipality": "Alavus",
    "population": 12103.0,
    "DL avg": 55538.953271028,
    "DL var": 2165521483.2902474,
    "UL avg": 33308.9252336449,
    "UL var": 1212724052.4283187
  },
  {
    "municipality": "Askola",
    "population": 5064.0,
    "DL avg": 43079.6666666667,
    "DL var": 3040475683.7692304,
    "UL avg": 8556.6666666667,
    "UL var": 59177589.76923076
  },
  {
    "municipality": "Aura",
    "population": 3982.0,
    "DL avg": 27335.247706422,
    "DL var": 545926695.1510361,
    "UL avg": 6654.6168224299,
    "UL var": 49107248.238582276
  }
]


In [69]:
combined_df.rename(columns = {"municipality": "name"}, inplace = True)

def row_to_xml(row):
    xml = ["\t<municipality>"]
    for i, col_name in enumerate(row.index):
        xml.append(f"\t\t<{col_name}>{row.iloc[i]}</{col_name}>")
    xml.append("\t</municipality>")
    return "\n".join(xml)

def df_to_xml(df, filename = None):    
    res = "\n".join(df.apply(row_to_xml, axis = 1))
    res = "<NetSpeedData>\n" + res + "\n</NetSpeedData>"
    
    if filename is not None:
        with open(filename, "w") as outfile:
            outfile.write(res)
    else:
        return res

df_to_xml(combined_df, "netspeeds.xml")

print(df_to_xml(combined_df)[0:965])

<NetSpeedData>
	<municipality>
		<name>Akaa</name>
		<population>17052.0</population>
		<DL avg>37535.52173913043</DL avg>
		<DL var>2982846676.341433</DL var>
		<UL avg>14488.376811594202</UL avg>
		<UL var>893404787.8853366</UL var>
	</municipality>
	<municipality>
		<name>Alajärvi</name>
		<population>10171.0</population>
		<DL avg>45620.145833333336</DL avg>
		<DL var>958465963.2761523</DL var>
		<UL avg>11303.9375</UL avg>
		<UL var>77204422.6981383</UL var>
	</municipality>
	<municipality>
		<name>Alavus</name>
		<population>12103.0</population>
		<DL avg>55538.95327102804</DL avg>
		<DL var>2165521483.2902474</DL var>
		<UL avg>33308.92523364486</UL avg>
		<UL var>1212724052.4283187</UL var>
	</municipality>
	<municipality>
		<name>Askola</name>
		<population>5064.0</population>
		<DL avg>43079.666666666664</DL avg>
		<DL var>3040475683.7692304</DL var>
		<UL avg>8556.666666666666</UL avg>
		<UL var>59177589.76923076</UL var>
	</municipality>
	
